In [3]:
season = Season.objects.get(id=3)
season

<Season: 3: 2019 Fall Start:2019-09-16 End:2020-01-05>

In [32]:
season_players = SeasonPlayer.objects.filter(season=season)
season_players.count()

guys = SeasonPlayer.objects.filter(season=season, player__gender='M')
girls = SeasonPlayer.objects.filter(season=season, player__gender='F')

In [90]:
Schedule.objects.filter(meeting__season=season).count()
Schedule.objects.filter(meeting__season=season).delete()

(7, {'blockdb.ScheduleVerify': 0, 'blockdb.Schedule': 7})

In [28]:
# Schedule some players
from django.db import IntegrityError
def add_to_schedule(player, meetings, plays):
    for i, did_play in enumerate(plays):
        if did_play:
            try:
                Schedule.objects.create(
                    meeting=meetings[i],
                    player=player
                )
            except IntegrityError:
                pass

In [61]:
Schedule.objects.filter(meeting__season=season).delete()
add_to_schedule(guys[0].player, season.meetings, [1,0,1,0,1,0, 1, 1, 0])
add_to_schedule(girls[0].player, season.meetings, [1,0,1,0,1,0, 0, 0, 1])

In [42]:
couple = Couple(male=guys[0].player, female=girls[0].player)

scheduled_meetings = Schedule.objects.filter(
    meeting__season=season
)

he = scheduled_meetings.filter(
    Q(player=couple.male)
).values_list('meeting__id', flat=True)

she = scheduled_meetings.filter(
    Q(player=couple.female)
).values_list('meeting__id', flat=True)

he_only = scheduled_meetings.filter(
    Q(player=couple.male) & ~Q(meeting__id__in=she)
).count()

she_only = scheduled_meetings.filter(
    Q(player=couple.female) & ~Q(meeting__id__in=he)
).count()

they = scheduled_meetings.filter(
    Q(player=couple.male) | Q(player=couple.female)
).count()

f"{he_only} {she_only} {they}"

'2 1 9'

In [75]:
from django.db.models import Q
f = Count('player', filter=Q(player__gender='F'))
m = Count('player', filter=Q(player__gender='M'))
stats = Schedule.objects.filter(
    meeting__season=season
).order_by('meeting__date').values('meeting').annotate(f=f).annotate(m=m)

In [78]:
stats

<QuerySet [{'meeting': 30, 'f': 1, 'm': 1}, {'meeting': 32, 'f': 1, 'm': 1}, {'meeting': 34, 'f': 1, 'm': 1}, {'meeting': 36, 'f': 0, 'm': 1}, {'meeting': 37, 'f': 0, 'm': 1}, {'meeting': 38, 'f': 1, 'm': 0}]>

In [87]:
they=0
he=0
she=0

for stat in stats:
    f = stat.get('f')
    m = stat.get('m')
    if f and m: they += 1
    if m and not f: he += 1
    if f and not m: she += 1

    
f"Both:{they} He:{he} She:{she}"

'Both:3 He:2 She:1'